# Chap 2. Prompt Principle

编写清晰、具体的指令，给予模型充分思考时间

## 1. Principle 1: 清晰、具体的指令

### 1.1. 分隔符表示输入的不同部分

In [1]:
from tool import get_completion

text = f"""
您应该提供尽可能清晰、具体的指示，以表达您希望模型执行的任务。\
这将引导模型朝向所需的输出，并降低收到无关或不正确响应的可能性。\
不要将写清晰的提示词与写简短的提示词混淆。\
在许多情况下，更长的提示词可以为模型提供更多的清晰度
"""

# 需要总结的文本内容
prompt = f"""
把用三个反引号括起来的文本总结成一句话。
```{text}```
"""

# 指令内容，使用 ``` 来分隔指令和待总结的内容
response = get_completion(prompt)
print(response)


好的，这句话可以总结为：

**为了获得更准确的相关回复，应向模型提供清晰、具体且内容详尽的指示。**


### 1.2. 结构化输出

In [2]:
prompt = f"""
请生成包括书名、作者和类别的三本虚构的、非真实存在的中文书籍清单，\
并以 JSON 格式提供，其中包含以下键:book_id、title、author、genre。
"""

response = get_completion(prompt)
print(response)

{
  "books": [
    {
      "book_id": "B001",
      "title": "量子玫瑰的回忆录",
      "author": "陈星云",
      "genre": "科幻小说"
    },
    {
      "book_id": "B002",
      "title": "长安雾雨图",
      "author": "林墨白",
      "genre": "历史悬疑"
    },
    {
      "book_id": "B003",
      "title": "心谷回音",
      "author": "吴明哲",
      "genre": "心理学"
    }
  ]
}


### 1.3. 检查条件满足

In [3]:
# 满足条件的输入（text中提供了步骤）

text_1 = f"""
泡一杯茶很容易。首先，需要把水烧开。\
在等待期间，拿一个杯子并把茶包放进去。\
一旦水足够热，就把它倒在茶包上。\
等待一会儿，让茶叶浸泡。几分钟后，取出茶包。\
如果您愿意，可以加一些糖或牛奶调味。\
就这样，您可以享受一杯美味的茶了。
"""

prompt = f"""
您将获得由三个引号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：

第一步 - ...
第二步 - …
…
第N步 - …

如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Text 1 的总结：")
print(response)

Text 1 的总结：
第一步 - 把水烧开。  
第二步 - 拿一个杯子并把茶包放进去。  
第三步 - 将烧开的水倒在茶包上。  
第四步 - 等待几分钟，让茶叶浸泡。  
第五步 - 取出茶包。  
第六步 - 根据个人喜好添加糖或牛奶调味。


In [4]:
# 不满足条件的输入（text中未提供预期指令）
text_2 = f"""
今天阳光明媚，鸟儿在歌唱。\
这是一个去公园散步的美好日子。\
鲜花盛开，树枝在微风中轻轻摇曳。\
人们外出享受着这美好的天气，有些人在野餐，有些人在玩游戏或者在草地上放松。\
这是一个完美的日子，可以在户外度过并欣赏大自然的美景。
"""

prompt = f"""
您将获得由三个引号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：

第一步 - ...
第二步 - …
…
第N步 - …

如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
\"\"\"{text_2}\"\"\"
"""

response = get_completion(prompt)
print("Text 2 的总结：")
print(response)


Text 2 的总结：
未提供步骤。


### 1.4. 提供少量实例

In [6]:
prompt = f"""
您的任务是以一致的风格回答问题。

<孩子>: 请教我何为耐心。

<祖父母>: 挖出最深峡谷的河流源于一处不起眼的泉眼；最宏伟的交响乐从单一的音符开始；最复杂的挂毯以一根孤独的线开始编织。

<孩子>: 请教我何为韧性。
"""

response = get_completion(prompt)
print(response)

<祖父母>: 竹子随风弯曲却不折断，寒冬中看似枯萎，待春来又发新枝；老树根须在岩石间蜿蜒，寻找每一丝缝隙，终成参天之势。水滴日夜叩击同一处石檐，竟能凿穿坚硬的青石板。


## 2. Principle 2: 给模型时间思考

### 2.1. 指定任务所需的步骤

In [7]:
text = f"""
在一个迷人的村庄里，兄妹杰克和吉尔出发去一个山顶井里打水。\
他们一边唱着欢乐的歌，一边往上爬，\
然而不幸降临——杰克绊了一块石头，从山上滚了下来，吉尔紧随其后。\
虽然略有些摔伤，但他们还是回到了温馨的家中。\
尽管出了这样的意外，他们的冒险精神依然没有减弱，继续充满愉悦地探索。
"""

# e.g. 1.
prompt_1 = f"""
执行以下操作：
1-用一句话概括下面用三个反引号括起来的文本。
2-将摘要翻译成英语。
3-在英语摘要中列出每个人名。
4-输出一个 JSON 对象，其中包含以下键：english_summary，num_names。

请用换行符分隔您的答案。

Text:
```{text}```
"""

response = get_completion(prompt_1)
print("prompt 1:")
print(response)

prompt 1:
在一个迷人的村庄里，兄妹杰克和吉尔前往山顶打水时遭遇意外滚落山坡，但最终安全返回家中，并未减弱他们的冒险精神。  
In a charming village, siblings Jack and Jill set out to fetch water from a hilltop well but accidentally tumble down the slope; they return home slightly injured yet undeterred in their adventurous spirit.  
Jack, Jill  
{"english_summary": "In a charming village, siblings Jack and Jill set out to fetch water from a hilltop well but accidentally tumble down the slope; they return home slightly injured yet undeterred in their adventurous spirit.", "num_names": 2}


In [9]:
prompt_2 = f"""
1-用一句话概括下面用<>括起来的文本。
2-将摘要翻译成英语。
3-在英语摘要中列出每个名称。
4-输出一个 JSON 对象，其中包含以下键：English_summary，num_names。

请使用以下格式：
文本：<要总结的文本>
摘要：<摘要>
翻译：<摘要的翻译>
名称：<英语摘要中的名称列表>
输出 JSON：<带有 English_summary 和 num_names 的 JSON>

Text: <{text}>
"""

response = get_completion(prompt_2)
print("\nprompt 2:")
print(response)


prompt 2:
文本：<在一个迷人的村庄里，兄妹杰克和吉尔出发去一个山顶井里打水。他们一边唱着欢乐的歌，一边往上爬，然而不幸降临——杰克绊了一块石头，从山上滚了下来，吉尔紧随其后。虽然略有些摔伤，但他们还是回到了温馨的家中。尽管出了这样的意外，他们的冒险精神依然没有减弱，继续充满愉悦地探索。>

摘要：杰克和吉尔兄妹打水时从山上滚落但未受伤，保持冒险精神继续探索。

翻译：Siblings Jack and Jill fell down a hill while fetching water but were unharmed and maintained their adventurous spirit to continue exploring.

名称：Jack, Jill

输出 JSON：{"English_summary": "Siblings Jack and Jill fell down a hill while fetching water but were unharmed and maintained their adventurous spirit to continue exploring.", "num_names": 2}


### 2.2. 直到模型找出自己解法

In [10]:
prompt = f"""
判断学生的解决方案是否正确。

问题:
我正在建造一个太阳能发电站，需要帮助计算财务。

  土地费用为 100美元/平方英尺
  我可以以 250美元/平方英尺的价格购买太阳能电池板
  我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
  作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：
设x为发电站的大小，单位为平方英尺。
费用：

  土地费用：100x
  太阳能电池板费用：250x
  维护费用：100,000美元+100x
  总费用：100x+250x+100,000美元+100x=450x+100,000美元
"""

response = get_completion(prompt)
print(response)

让我们逐步分析学生的解决方案是否正确。

**问题陈述：**
- 土地费用：100美元/平方英尺
- 太阳能电池板费用：250美元/平方英尺
- 维护费用：每年固定10万美元，加上每平方英尺10美元
- 需要求：首年运营的总费用作为平方英尺数（x）的函数。

**学生的解决方案：**
- 土地费用：100x
- 太阳能电池板费用：250x
- 维护费用：100,000 + 100x   （注意：这里学生写了100x，但应该是10x）
- 总费用：100x + 250x + 100,000 + 100x = 450x + 100,000

**错误点：**
维护费用中，每平方英尺是10美元，而不是100美元。因此，维护费用应为：100,000 + 10x。

**正确的总费用计算：**
- 土地费用：100x
- 太阳能电池板费用：250x
- 维护费用：100,000 + 10x
- 总费用：100x + 250x + 100,000 + 10x = (100 + 250 + 10)x + 100,000 = 360x + 100,000

**结论：**
学生的解决方案中，维护费用的可变部分错误地使用了100x（应该是10x），导致总费用中x的系数错误（450x而不是正确的360x）。因此，学生的解决方案是不正确的。

**正确解答：**
首年运营的总费用为 \( 360x + 100,000 \) 美元，其中x是发电站的大小（平方英尺）。


In [11]:
prompt = f"""
请判断学生的解决方案是否正确，请通过如下步骤解决这个问题：

步骤：

  首先，自己解决问题。
  然后将您的解决方案与学生的解决方案进行比较，对比计算得到的总费用与学生计算的总费用是否一致，并评估学生的解决方案是否正确。
  在自己完成问题之前，请勿决定学生的解决方案是否正确。

使用以下格式：

  问题：问题文本
  学生的解决方案：学生的解决方案文本
  实际解决方案和步骤：实际解决方案和步骤文本
  学生计算的总费用：学生计算得到的总费用
  实际计算的总费用：实际计算出的总费用
  学生计算的费用和实际计算的费用是否相同：是或否
  学生的解决方案和实际解决方案是否相同：是或否
  学生的成绩：正确或不正确

问题：

  我正在建造一个太阳能发电站，需要帮助计算财务。 
  - 土地费用为每平方英尺100美元
  - 我可以以每平方英尺250美元的价格购买太阳能电池板
  - 我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元;

  作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：

  设x为发电站的大小，单位为平方英尺。
  费用：
  1. 土地费用：100x美元
  2. 太阳能电池板费用：250x美元
  3. 维护费用：100,000+100x=10万美元+10x美元
  总费用：100x美元+250x美元+10万美元+100x美元=450x+10万美元

实际解决方案和步骤：
"""

response = get_completion(prompt)
print(response)

首先，自己解决问题：

问题：计算首年运营的总费用作为平方英尺数x的函数。
- 土地费用：每平方英尺100美元，因此为100x美元。
- 太阳能电池板费用：每平方英尺250美元，因此为250x美元。
- 维护费用：每年固定10万美元，加上每平方英尺10美元，因此为100,000 + 10x美元。

总费用 = 土地费用 + 太阳能电池板费用 + 维护费用
= 100x + 250x + (100,000 + 10x)
合并同类项：
= (100x + 250x + 10x) + 100,000
= 360x + 100,000

因此，首年总费用为360x + 100,000美元。

现在，与学生的解决方案比较：
学生的解决方案：
总费用：100x + 250x + 100,000 + 100x = 450x + 100,000

学生错误地将维护费用中的每平方英尺10美元写成了100美元（即100x而不是10x）。因此，学生计算的总费用是450x + 100,000，而实际应为360x + 100,000。

学生计算的总费用：450x + 100,000
实际计算的总费用：360x + 100,000
两者不相同。

学生的解决方案和实际解决方案不相同，因为学生错误地使用了100x而不是10x表示维护费用的可变部分。

因此，学生的成绩为不正确。

最终回答：

问题：我正在建造一个太阳能发电站，需要帮助计算财务。 
- 土地费用为每平方英尺100美元
- 我可以以每平方英尺250美元的价格购买太阳能电池板
- 我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元;
作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：设x为发电站的大小，单位为平方英尺。
费用：
1. 土地费用：100x美元
2. 太阳能电池板费用：250x美元
3. 维护费用：100,000+100x=10万美元+10x美元
总费用：100x美元+250x美元+10万美元+100x美元=450x+10万美元

实际解决方案和步骤：
总费用 = 土地费用 + 太阳能电池板费用 + 维护费用
= 100x + 250x + (100,000 + 10x)
= (100x + 250x + 10x) + 100,000
= 360x + 100,000美元

## 3. 局限性

In [12]:
prompt = f"""
告诉我华为公司生产的GT Watch运动手表的相关信息
"""
response = get_completion(prompt)
print(response)


好的，很高兴为您介绍华为公司生产的GT Watch系列智能运动手表。这是一个非常成功和受欢迎的产品线，以其**长续航**和**专业运动健康功能**而闻名。

请注意，“GT Watch”是一个系列的总称，旗下包含了多代和多个型号的产品（如GT 1, 2, 3, 4, GT Runner等）。下面我将从系列共性、主要型号差异、核心功能和特点等方面为您进行全面介绍。

### 一、系列核心共性特点

1.  **超长续航**：这是GT系列最大的卖点。与传统智能手表（如Apple Watch需要每天一充）不同，GT系列在典型使用模式下通常可以达到**7-14天**的续航，重度使用也能支撑数天，基础手表模式甚至可达数周。这得益于其低功耗芯片和优化的操作系统。
2.  **专业健康监测**：
    *   **华为TruSeen™心率监测技术**：提供连续、精准的心率监测。
    *   **血氧饱和度（SpO2）监测**：支持单次和连续血氧监测，尤其适合在高强度运动、高海拔活动或睡眠时评估身体状态。
    *   **睡眠监测**：提供详细的睡眠结构分析（深睡、浅睡、REM快速眼动睡眠）、睡眠评分和改善建议。
    *   **压力监测**：通过心率变异性（HRV）分析压力水平，并提供呼吸训练进行放松。
    *   **女性健康管理**：记录生理周期并提供预测提醒。
3.  **强大的运动功能**：
    *   内置**100+种运动模式**，覆盖主流、户外、室内、水上等各种运动类型。
    *   支持**多种卫星定位系统**（GPS、GLONASS、Galileo、北斗），提供精准的运动轨迹记录。
    *   为跑步者提供专业数据（如最大摄氧量VO₂Max、恢复时间建议、训练负荷、配速等）。
4.  **智能体验**：
    *   **蓝牙通话**：大部分型号支持通过手表直接接听或拨打电话（需与手机蓝牙连接）。
    *   **音乐播放**：内置存储空间，可独立存储和播放音乐，运动时无需手机。
    *   **NFC功能**：支持华为钱包，用于公交卡、门禁卡模拟和移动支付（支付宝/微信支付二维码）。
    *   **消息通知**：显示手机上的来电、短信、App通知（可查看，但无法复杂回复）。

### 二、主要型号差异与演进

华为